# Section II
Hydrology
Validate your installations, in a notebook by executing:

!pip install earthengine-api geemap folium matplotlib pandas geopandas rasterio rasterstats scipy scikit-learn seaborn statsmodels !earthengine authenticate



In [1]:
# Replace by your project numbber:
my_project = 'my-project-0000000000000000'

In [2]:
import ee
from datetime import datetime, timezone
# Initialize Earth Engine
try:
    ee.Initialize(project=my_project)
    print("Earth Engine initialized successfully!")
except Exception as e:
    print(f"Error initializing Earth Engine: {e}")
    print("Please authenticate using: ee.Authenticate()")

Earth Engine initialized successfully!


In [3]:
def comprehensive_surface_water_mapping(region, date_range, confidence_threshold=0.8):
    """
    Creates comprehensive surface water maps using multi-sensor fusion.
    Combines Landsat, Sentinel-1 SAR, and auxiliary datasets.
    """
    
    start_date, end_date = date_range
    
    # 1. Optical water detection using multiple indices
    def optical_water_detection(image):
        """Enhanced optical water detection using multiple spectral indices."""
        
        # Calculate water indices
        green = image.select('SR_B3').multiply(0.0000275).add(-0.2)
        nir = image.select('SR_B5').multiply(0.0000275).add(-0.2)
        swir1 = image.select('SR_B6').multiply(0.0000275).add(-0.2)
        swir2 = image.select('SR_B7').multiply(0.0000275).add(-0.2)
        
        # Multiple water indices for robust detection
        ndwi = green.subtract(nir).divide(green.add(nir)).rename('NDWI')
        mndwi = green.subtract(swir1).divide(green.add(swir1)).rename('MNDWI')
        awesh = green.add(swir1.multiply(2.5)).subtract(nir.multiply(1.5)).subtract(swir2.multiply(0.25)).rename('AWESH')
        
        # Combine indices using weighted approach
        water_probability = ndwi.multiply(0.4) \
            .add(mndwi.multiply(0.4)) \
            .add(awesh.multiply(0.2)) \
            .rename('water_probability_optical')
        
        # Apply dynamic thresholds based on local conditions
        water_mask_optical = water_probability.gt(0.3)
        
        return image.addBands([water_probability, water_mask_optical])
    
    # Process Landsat collection
    landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterBounds(region) \
        .filterDate(start_date, end_date) \
        .map(advanced_cloud_mask) \
        .map(optical_water_detection)
    
    optical_water_composite = landsat_collection.median()
    
    # 2. SAR-based water detection
    def sar_water_detection(image):
        """Water detection using Sentinel-1 SAR data."""
        
        # VV polarization is most sensitive to water
        vv = image.select('VV')
        vh = image.select('VH')
        
        # Apply speckle filtering
        vv_filtered = vv.focalMean(1, 'square', 'pixels')
        vh_filtered = vh.focalMean(1, 'square', 'pixels')
        
        # Water typically has very low backscatter in VV
        water_mask_vv = vv_filtered.lt(-18)  # Threshold in dB
        
        # Cross-polarization ratio for additional discrimination
        vh_vv_ratio = vh_filtered.subtract(vv_filtered)
        water_mask_ratio = vh_vv_ratio.lt(-8)
        
        # Combine SAR indicators
        water_probability_sar = ee.Image(0) \
            .where(water_mask_vv.And(water_mask_ratio), 1) \
            .where(water_mask_vv.Or(water_mask_ratio), 0.5) \
            .rename('water_probability_sar')
        
        return image.addBands(water_probability_sar)
    
    # Process Sentinel-1 collection
    sentinel1_collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(region) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .map(sar_water_detection)
    
    sar_water_composite = sentinel1_collection.median()
    
    # 3. Multi-sensor fusion
    def fuse_water_detections(optical_prob, sar_prob, dem, slope):
        """
        Fuses optical and SAR water detections using topographic context.
        """
        
        # Weight SAR more heavily in areas prone to optical limitations
        # (e.g., areas with frequent clouds, shadows)
        optical_weight = ee.Image(0.6)  # Base weight for optical
        sar_weight = ee.Image(0.4)      # Base weight for SAR
        
        # Adjust weights based on topographic context
        # SAR performs better in flat areas, optical in steep terrain
        flat_areas = slope.lt(5)  # Degrees
        optical_weight = optical_weight.where(flat_areas, 0.4)
        sar_weight = sar_weight.where(flat_areas, 0.6)
        
        # Combine probabilities
        fused_probability = optical_prob.multiply(optical_weight) \
            .add(sar_prob.multiply(sar_weight))
        
        # Apply topographic constraints
        # Water unlikely above certain elevation or on steep slopes
        elevation_mask = dem.lt(3000)  # Meters above sea level
        slope_mask = slope.lt(15)      # Degrees
        topographic_constraint = elevation_mask.And(slope_mask)
        
        final_probability = fused_probability.updateMask(topographic_constraint)
        final_water_mask = final_probability.gt(confidence_threshold)
        
        return {
            'water_probability': final_probability,
            'water_mask': final_water_mask,
            'fusion_weights': ee.Image.cat([optical_weight, sar_weight]).rename(['optical_weight', 'sar_weight'])
        }
    
    # Get topographic data
    dem = ee.Image('USGS/SRTMGL1_003')
    slope = ee.Terrain.slope(dem)
    
    # Perform fusion
    fusion_result = fuse_water_detections(
        optical_water_composite.select('water_probability_optical'),
        sar_water_composite.select('water_probability_sar'),
        dem,
        slope
    )
    
    return fusion_result

# 4. Temporal water dynamics analysis
def analyze_water_dynamics(water_time_series, region):
    """
    Analyzes temporal patterns in surface water extent.
    """
    
    # Calculate water occurrence frequency
    water_occurrence = water_time_series.select('water_mask').mean().rename('water_occurrence')
    
    # Classify water persistence
    permanent_water = water_occurrence.gt(0.8)
    seasonal_water = water_occurrence.gt(0.2).And(water_occurrence.lte(0.8))
    ephemeral_water = water_occurrence.gt(0.05).And(water_occurrence.lte(0.2))
    
    # Calculate seasonal patterns
    def seasonal_analysis(collection):
        seasons = {
            'spring': [3, 4, 5],
            'summer': [6, 7, 8], 
            'autumn': [9, 10, 11],
            'winter': [12, 1, 2]
        }
        
        seasonal_water = {}
        for season, months in seasons.items():
            seasonal_collection = collection.filter(
                ee.Filter.calendarRange(months[0], months[-1], 'month')
            )
            seasonal_water[season] = seasonal_collection.select('water_mask').mean()
        
        return seasonal_water
    
    seasonal_patterns = seasonal_analysis(water_time_series)
    
    # Calculate water extent time series
    def calculate_water_extent(image):
        extent = image.select('water_mask').reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=region,
            scale=30,
            maxPixels=1e9
        ).get('water_mask')
        
        return ee.Feature(None, {
            'date': image.date().format('YYYY-MM-dd'),
            'water_extent_pixels': extent,
            'water_extent_hectares': ee.Number(extent).multiply(0.09)  # 30m pixel = 0.09 ha
        })
    
    extent_time_series = water_time_series.map(calculate_water_extent)
    
    return {
        'water_occurrence': water_occurrence,
        'permanent_water': permanent_water,
        'seasonal_water': seasonal_water,
        'ephemeral_water': ephemeral_water,
        'seasonal_patterns': seasonal_patterns,
        'extent_time_series': extent_time_series
    }

In [4]:
def comprehensive_precipitation_analysis(region, analysis_period, reference_stations=None):
    """
    Comprehensive precipitation analysis combining satellite data with ground observations.
    """
    
    start_date, end_date = analysis_period
    
    # 1. Multi-source precipitation data integration
    def integrate_precipitation_sources():
        """Integrates multiple precipitation datasets for robust analysis."""
        
        # CHIRPS - Climate Hazards Group InfraRed Precipitation with Station data
        chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY') \
            .filterDate(start_date, end_date) \
            .filterBounds(region)
        
        # GPM IMERG - Global Precipitation Measurement Integrated Multi-satellitE Retrievals
        gpm = ee.ImageCollection('NASA/GPM_L3/IMERG_V06') \
            .filterDate(start_date, end_date) \
            .filterBounds(region) \
            .select('precipitationCal')
        
        # TRMM (if analyzing historical period)
        trmm = ee.ImageCollection('TRMM/3B42') \
            .filterDate(start_date, end_date) \
            .filterBounds(region) \
            .select('precipitation')
        
        # Bias correction and harmonization
        def harmonize_precipitation_data(chirps_col, gpm_col):
            """Harmonizes different precipitation products."""
            
            # Resample GPM to CHIRPS resolution (0.05°)
            def resample_gpm(image):
                return image.resample('bilinear').reproject(
                    crs=chirps_col.first().projection(),
                    scale=5566  # CHIRPS native resolution
                )
            
            gpm_resampled = gpm_col.map(resample_gpm)
            
            # Calculate bias correction factors
            def calculate_bias_correction():
                chirps_mean = chirps_col.mean()
                gpm_mean = gpm_resampled.mean()
                bias_factor = chirps_mean.divide(gpm_mean)
                return bias_factor
            
            bias_correction = calculate_bias_correction()
            
            # Apply bias correction to GPM
            def apply_bias_correction(image):
                return image.multiply(bias_correction).copyProperties(image, ['system:time_start'])
            
            gpm_corrected = gpm_resampled.map(apply_bias_correction)
            
            return chirps_col, gpm_corrected
        
        chirps_harmonized, gpm_harmonized = harmonize_precipitation_data(chirps, gpm)
        
        return {
            'chirps': chirps_harmonized,
            'gpm': gpm_harmonized,
            'trmm': trmm
        }
    
    precipitation_sources = integrate_precipitation_sources()
    
    # 2. Extreme precipitation event detection
    def detect_extreme_precipitation_events(precip_collection, return_periods=[2, 5, 10, 25]):
        """
        Detects extreme precipitation events using statistical analysis.
        """
        
        # Calculate daily precipitation statistics
        daily_stats = precip_collection.map(lambda img: 
            img.reduceRegion(
                reducer=ee.Reducer.mean().combine(ee.Reducer.max(), sharedInputs=True),
                geometry=region,
                scale=5566,
                maxPixels=1e8
            ).set('date', img.date().format('YYYY-MM-dd'))
        )
        
        # Calculate precipitation percentiles for extreme event thresholds
        precipitation_percentiles = precip_collection.reduce(ee.Reducer.percentile([90, 95, 99, 99.9]))
        
        # Identify extreme events
        def classify_extreme_events(image):
            p90 = precipitation_percentiles.select('precipitation_p90')
            p95 = precipitation_percentiles.select('precipitation_p95')
            p99 = precipitation_percentiles.select('precipitation_p99')
            
            moderate_extreme = image.gt(p90).And(image.lte(p95))
            severe_extreme = image.gt(p95).And(image.lte(p99))
            extreme_extreme = image.gt(p99)
            
            return image.addBands([
                moderate_extreme.rename('moderate_extreme'),
                severe_extreme.rename('severe_extreme'),
                extreme_extreme.rename('extreme_extreme')
            ])
        
        extreme_events = precip_collection.map(classify_extreme_events)
        
        return {
            'extreme_events': extreme_events,
            'percentile_thresholds': precipitation_percentiles,
            'daily_stats': daily_stats
        }
    
    extreme_analysis = detect_extreme_precipitation_events(precipitation_sources['chirps'])
    
    # 3. Drought monitoring using standardized precipitation index (SPI)
    def calculate_standardized_precipitation_index(precip_collection, time_scales=[1, 3, 6, 12]):
        """
        Calculates SPI for multiple time scales for drought monitoring.
        """
        
        spi_results = {}
        
        for time_scale in time_scales:
            # Calculate rolling sum for the time scale (in months)
            def calculate_rolling_precipitation(image):
                # Get date of current image
                current_date = image.date()
                
                # Calculate start date for rolling window
                start_date = current_date.advance(-time_scale, 'month')
                
                # Filter collection for rolling window
                rolling_collection = precip_collection.filterDate(start_date, current_date)
                
                # Sum precipitation over window
                rolling_sum = rolling_collection.sum()
                
                return rolling_sum.set('date', current_date.format('YYYY-MM-dd'))
            
            # Calculate rolling precipitation
            rolling_precip = precip_collection.map(calculate_rolling_precipitation)
            
            # Calculate SPI (simplified - assumes gamma distribution)
            precip_mean = rolling_precip.mean()
            precip_std = rolling_precip.reduce(ee.Reducer.stdDev())
            
            def calculate_spi(image):
                # Standardize precipitation values
                spi = image.subtract(precip_mean).divide(precip_std)
                return spi.rename(f'SPI_{time_scale}month')
            
            spi_collection = rolling_precip.map(calculate_spi)
            spi_results[f'{time_scale}_month'] = spi_collection
        
        return spi_results
    
    spi_analysis = calculate_standardized_precipitation_index(precipitation_sources['chirps'])
    
    # 4. Hydrological modeling integration
    def integrate_hydrological_modeling(precipitation_data, catchment_geometry):
        """
        Integrates precipitation data with simple hydrological modeling.
        """
        
        # Get additional data for hydrological modeling
        dem = ee.Image('USGS/SRTMGL1_003')
        slope = ee.Terrain.slope(dem)
        
        # Land cover for runoff coefficient estimation
        landcover = ee.Image('ESA/WorldCover/v100/2020')
        
        # Soil data (example using global datasets)
        soil_properties = ee.Image('OpenLandMap/SOL/SOL_WATERCONTENT-33KPA_USDA-4A1H_M/v01') \
            .select('b0')  # Water content at field capacity
        
        # Simple runoff calculation using SCS Curve Number method (simplified)
        def calculate_runoff_coefficient(landcover_img, soil_img, slope_img):
            """Calculate runoff coefficient based on land cover and soil properties."""
            
            # Simplified curve numbers for different land covers
            curve_numbers = ee.Image(0) \
                .where(landcover_img.eq(10), 30) \
                .where(landcover_img.eq(20), 55) \
                .where(landcover_img.eq(40), 70) \
                .where(landcover_img.eq(50), 85) \
                .where(landcover_img.eq(80), 95) \
                .rename('curve_number')
            
            # Adjust for slope (higher slopes = higher runoff)
            slope_factor = slope_img.divide(45).multiply(0.2).add(0.8)  # Range 0.8-1.0
            
            adjusted_cn = curve_numbers.multiply(slope_factor)
            runoff_coefficient = adjusted_cn.divide(100)
            
            return runoff_coefficient
        
        runoff_coeff = calculate_runoff_coefficient(landcover, soil_properties, slope)
        
        # Calculate potential runoff
        def calculate_runoff(precip_image):
            potential_runoff = precip_image.multiply(runoff_coeff)
            
            # Apply initial abstraction (simplified)
            initial_abstraction = runoff_coeff.multiply(-5)  # mm
            actual_runoff = potential_runoff.add(initial_abstraction).max(0)
            
            return precip_image.addBands([
                potential_runoff.rename('potential_runoff'),
                actual_runoff.rename('actual_runoff')
            ])
        
        runoff_collection = precipitation_sources['chirps'].map(calculate_runoff)
        
        # Calculate catchment-scale runoff
        def calculate_catchment_runoff(image):
            catchment_runoff = image.select('actual_runoff').reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=catchment_geometry,
                scale=5566,
                maxPixels=1e8
            )
            
            return ee.Feature(None, {
                'date': image.date().format('YYYY-MM-dd'),
                'mean_runoff_mm': catchment_runoff.get('actual_runoff')
            })
        
        catchment_runoff_series = runoff_collection.map(calculate_catchment_runoff)
        
        return {
            'runoff_collection': runoff_collection,
            'runoff_coefficient': runoff_coeff,
            'catchment_runoff_series': catchment_runoff_series
        }
    
    # Apply hydrological modeling if catchment geometry is provided
    if region.type().getInfo() == 'Polygon':
        hydrological_analysis = integrate_hydrological_modeling(
            precipitation_sources['chirps'], 
            region
        )
    else:
        hydrological_analysis = None
    
    return {
        'precipitation_sources': precipitation_sources,
        'extreme_analysis': extreme_analysis,
        'spi_analysis': spi_analysis,
        'hydrological_analysis': hydrological_analysis
    }

In [5]:
def comprehensive_climate_analysis(region, analysis_period, climate_variables=None):
    """
    Comprehensive climate analysis using multiple reanalysis products.
    """
    
    if climate_variables is None:
        climate_variables = [
            'temperature', 'precipitation', 'humidity', 'wind', 
            'pressure', 'radiation', 'evapotranspiration'
        ]
    
    start_date, end_date = analysis_period
    
    # Define climate data sources
    climate_datasets = {
        'era5_monthly': 'ECMWF/ERA5/MONTHLY',
        'era5_daily': 'ECMWF/ERA5_LAND/DAILY_AGGR',
        'terraclimate': 'IDAHO_EPSCOR/TERRACLIMATE',
        'gridmet': 'IDAHO_EPSCOR/GRIDMET',
        'chirps': 'UCSB-CHG/CHIRPS/DAILY'
    }
    
    def extract_comprehensive_climate_data():
        """Extract and harmonize climate variables from multiple sources."""
        
        climate_data = {}
        
        # ERA5 Monthly - High-quality reanalysis for long-term trends
        if 'temperature' in climate_variables or 'humidity' in climate_variables:
            era5_monthly = ee.ImageCollection('ECMWF/ERA5/MONTHLY') \
                .filterDate(start_date, end_date) \
                .filterBounds(region)
            
            climate_data['era5_monthly'] = {
                'collection': era5_monthly,
                'variables': {
                    'temperature_2m': 'mean_2m_air_temperature',
                    'temperature_max': 'maximum_2m_air_temperature_since_previous_post_processing',
                    'temperature_min': 'minimum_2m_air_temperature_since_previous_post_processing',
                    'dewpoint': 'dewpoint_2m_temperature',
                    'humidity': 'relative_humidity_2m',
                    'pressure': 'surface_pressure',
                    'wind_u': '10m_u_component_of_wind',
                    'wind_v': '10m_v_component_of_wind'
                }
            }
        
        # TerraClimate - High resolution climate data
        terraclimate = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE') \
            .filterDate(start_date, end_date) \
            .filterBounds(region)
        
        climate_data['terraclimate'] = {
            'collection': terraclimate,
            'variables': {
                'temperature_max': 'tmmx',
                'temperature_min': 'tmmn',
                'precipitation': 'pr',
                'vapor_pressure_deficit': 'vpd',
                'wind_speed': 'vs',
                'soil_moisture': 'soil',
                'evapotranspiration': 'aet',
                'potential_evapotranspiration': 'pet'
            }
        }
        
        # GridMET - High resolution meteorological data for continental US
        if region.bounds().getInfo()['coordinates'][0][0][0] > -180:  # Check if in Americas
            gridmet = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET') \
                .filterDate(start_date, end_date) \
                .filterBounds(region)
            
            climate_data['gridmet'] = {
                'collection': gridmet,
                'variables': {
                    'temperature_max': 'tmmx',
                    'temperature_min': 'tmmn',
                    'precipitation': 'pr',
                    'humidity': 'rmax',
                    'wind_speed': 'vs',
                    'solar_radiation': 'srad',
                    'vapor_pressure_deficit': 'vpd',
                    'evapotranspiration': 'etr'
                }
            }
        
        return climate_data
    
    climate_data_sources = extract_comprehensive_climate_data()
    
    # Climate trend analysis
    def analyze_climate_trends(data_sources):
        """
        Analyzes long-term climate trends using multiple statistical methods.
        """
        
        trend_results = {}
        
        for source_name, source_data in data_sources.items():
            collection = source_data['collection']
            variables = source_data['variables']
            
            source_trends = {}
            
            for var_name, band_name in variables.items():
                if var_name not in climate_variables:
                    continue
                
                # Extract time series for the variable
                def extract_regional_mean(image):
                    # Handle unit conversions for different datasets
                    band = image.select(band_name)
                    
                    # Convert temperature from Kelvin to Celsius if needed
                    if 'temperature' in var_name and source_name == 'era5_monthly':
                        band = band.subtract(273.15)
                    
                    # Convert precipitation to mm/day if needed
                    if var_name == 'precipitation' and source_name == 'terraclimate':
                        band = band.multiply(0.1)  # TerraClimate precipitation is in mm/month

                    scaling = 27830 if source_name == 'era5_monthly' else 4638
                    
                    regional_mean = band.reduceRegion(
                        reducer=ee.Reducer.mean(),
                        geometry=region,
                        scale=scaling,  # Different scales for different datasets
                        maxPixels=1e12
                    ).get(band_name)
                    
                    return ee.Feature(None, {
                        'date': image.date().format('YYYY-MM-dd'),
                        'timestamp': image.get('system:time_start'),
                        'value': regional_mean,
                        'variable': var_name,
                        'source': source_name
                    })
                
                time_series = collection.map(extract_regional_mean)
                
                # Calculate trend statistics using linear regression
                def calculate_trend_statistics(time_series_fc):
                    """Calculate linear trend and statistical significance."""
                    
                    # This would typically be done client-side with more sophisticated statistics
                    # Here we show the server-side approach for trend detection
                    
                    # Convert to array for linear regression
                    time_series_array = time_series_fc.map(lambda f: 
                        ee.Feature(None, {
                            'x': ee.Number(f.get('timestamp')).divide(1000*60*60*24*365.25),  # Years since epoch
                            'y': f.get('value')
                        })
                    )
                    
                    # Simple linear regression (simplified)
                    x_values = time_series_array.aggregate_array('x')
                    y_values = time_series_array.aggregate_array('y')
                    
                    # Calculate correlation coefficient
                    correlation = ee.Array(x_values).subtract(ee.Array(x_values).reduce(ee.Reducer.mean(), [0])) \
                        .multiply(ee.Array(y_values).subtract(ee.Array(y_values).reduce(ee.Reducer.mean(), [0]))) \
                        .reduce(ee.Reducer.sum(), [0])
                    
                    return {
                        'time_series': time_series_array,
                        'trend_direction': correlation.gt(0)
                    }
                
                trend_stats = calculate_trend_statistics(time_series)
                source_trends[var_name] = {
                    'time_series': time_series,
                    'trend_statistics': trend_stats
                }
            
            trend_results[source_name] = source_trends
        
        return trend_results
    
    trend_analysis = analyze_climate_trends(climate_data_sources)
    
    # Climate extreme analysis
    def analyze_climate_extremes(data_sources):
        """
        Analyzes climate extremes including heat waves, cold spells, and extreme precipitation.
        """
        
        extreme_results = {}
        
        for source_name, source_data in data_sources.items():
            collection = source_data['collection']
            variables = source_data['variables']
            
            # Temperature extremes
            if 'temperature_max' in variables and 'temperature_min' in variables:
                temp_max_band = variables['temperature_max']
                temp_min_band = variables['temperature_min']
                
                # Calculate temperature thresholds (90th and 10th percentiles)
                temp_max_p90 = collection.select(temp_max_band).reduce(ee.Reducer.percentile([90]))
                temp_min_p10 = collection.select(temp_min_band).reduce(ee.Reducer.percentile([10]))
                
                # Identify extreme temperature events
                def identify_temperature_extremes(image):
                    temp_max = image.select(temp_max_band)
                    temp_min = image.select(temp_min_band)
                    
                    heat_wave_day = temp_max.gt(temp_max_p90)
                    cold_spell_day = temp_min.lt(temp_min_p10)
                    
                    return image.addBands([
                        heat_wave_day.rename('heat_wave_day'),
                        cold_spell_day.rename('cold_spell_day')
                    ])
                
                extreme_temp_collection = collection.map(identify_temperature_extremes)
                
                extreme_results[f'{source_name}_temperature_extremes'] = {
                    'collection': extreme_temp_collection,
                    'thresholds': {
                        'heat_wave_threshold': temp_max_p90,
                        'cold_spell_threshold': temp_min_p10
                    }
                }
            
            # Precipitation extremes
            if 'precipitation' in variables:
                precip_band = variables['precipitation']
                
                # Calculate extreme precipitation thresholds
                precip_thresholds = collection.select(precip_band).reduce(
                    ee.Reducer.percentile([95, 99, 99.9])
                )
                
                def identify_precipitation_extremes(image):
                    precip = image.select(precip_band)
                    
                    extreme_precip_p95 = precip.gt(precip_thresholds.select(f'{precip_band}_p95'))
                    extreme_precip_p99 = precip.gt(precip_thresholds.select(f'{precip_band}_p99'))
                    extreme_precip_p999 = precip.gt(precip_thresholds.select(f'{precip_band}_p99_9'))
                    
                    return image.addBands([
                        extreme_precip_p95.rename('extreme_precip_p95'),
                        extreme_precip_p99.rename('extreme_precip_p99'),
                        extreme_precip_p999.rename('extreme_precip_p999')
                    ])
                
                extreme_precip_collection = collection.map(identify_precipitation_extremes)
                
                extreme_results[f'{source_name}_precipitation_extremes'] = {
                    'collection': extreme_precip_collection,
                    'thresholds': precip_thresholds
                }
        
        return extreme_results
    
    extreme_analysis = analyze_climate_extremes(climate_data_sources)
    
    # Climate change impact assessment
    def assess_climate_change_impacts(trend_data, extreme_data, baseline_period=None):
        """
        Assesses climate change impacts using trend and extreme event analysis.
        """
        
        if baseline_period is None:
            baseline_period = ('1981-01-01', '2010-12-31')
        
        impact_assessment = {}
        
        # Temperature change assessment
        temperature_trends = []
        for source in trend_data:
            for variable in trend_data[source]:
                if 'temperature' in variable:
                    temperature_trends.append(trend_data[source][variable])
        
        # Precipitation change assessment
        precipitation_trends = []
        for source in trend_data:
            for variable in trend_data[source]:
                if 'precipitation' in variable:
                    precipitation_trends.append(trend_data[source][variable])
        
        # Extreme event frequency changes
        extreme_frequency_changes = {}
        for extreme_type in extreme_data:
            collection = extreme_data[extreme_type]['collection']
            
            # Calculate frequency during baseline vs. recent period
            baseline_collection = collection.filterDate(baseline_period[0], baseline_period[1])
            recent_collection = collection.filterDate('2010-01-01', end_date)
            
            # This would involve more detailed statistical analysis in practice
            extreme_frequency_changes[extreme_type] = {
                'baseline_collection': baseline_collection,
                'recent_collection': recent_collection
            }
        
        impact_assessment = {
            'temperature_trends': temperature_trends,
            'precipitation_trends': precipitation_trends,
            'extreme_frequency_changes': extreme_frequency_changes
        }
        
        return impact_assessment
    
    impact_assessment = assess_climate_change_impacts(trend_analysis, extreme_analysis)
    
    return {
        'climate_data_sources': climate_data_sources,
        'trend_analysis': trend_analysis,
        'extreme_analysis': extreme_analysis,
        'impact_assessment': impact_assessment
    }

In [6]:
def create_extreme_event_monitoring_system(region, monitoring_variables=None):
    """
    Creates an integrated system for monitoring multiple types of extreme events.
    """
    
    if monitoring_variables is None:
        monitoring_variables = ['flood', 'drought', 'heat_wave', 'extreme_precipitation']
    
    monitoring_system = {}
    
    # 1. Multi-hazard drought monitoring
    def setup_drought_monitoring():
        """
        Sets up comprehensive drought monitoring using multiple indicators.
        """
        
        # Palmer Drought Severity Index (PDSI) from TerraClimate
        terraclimate = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE')
        pdsi = terraclimate.select('pdsi')
        
        # Standardized Precipitation Index (SPI) - calculated from CHIRPS
        chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')
        
        # Vegetation health from MODIS
        modis_ndvi = ee.ImageCollection('MODIS/006/MOD13A1').select('NDVI')
        
        # Soil moisture from ERA5
        era5_land = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR')
        soil_moisture = era5_land.select('volumetric_soil_water_layer_1')
        
        def calculate_drought_composite_index(date):
            """
            Calculates composite drought index combining multiple indicators.
            """
            
            # Get data for the specified date (monthly composites)
            target_date = ee.Date(date)
            month_start = target_date.advance(-1, 'month')
            
            # PDSI (monthly)
            pdsi_value = pdsi.filterDate(month_start, target_date).mean()
            
            # SPI (3-month)
            spi_3m = chirps.filterDate(target_date.advance(-3, 'month'), target_date) \
                .sum().subtract(
                    chirps.filterDate('1981-01-01', '2010-12-31')
                    .filter(ee.Filter.calendarRange(target_date.get('month'), target_date.get('month'), 'month'))
                    .sum().reduce(ee.Reducer.mean())
                ).divide(
                    chirps.filterDate('1981-01-01', '2010-12-31')
                    .filter(ee.Filter.calendarRange(target_date.get('month'), target_date.get('month'), 'month'))
                    .sum().reduce(ee.Reducer.stdDev())
                )
            
            # Vegetation health anomaly
            ndvi_current = modis_ndvi.filterDate(month_start, target_date).mean()
            ndvi_climatology = modis_ndvi.filterDate('2001-01-01', '2020-12-31') \
                .filter(ee.Filter.calendarRange(target_date.get('month'), target_date.get('month'), 'month')) \
                .mean()
            ndvi_anomaly = ndvi_current.subtract(ndvi_climatology).divide(
                modis_ndvi.filterDate('2001-01-01', '2020-12-31') \
                .filter(ee.Filter.calendarRange(target_date.get('month'), target_date.get('month'), 'month')) \
                .reduce(ee.Reducer.stdDev())
            )
            
            # Soil moisture anomaly
            sm_current = soil_moisture.filterDate(month_start, target_date).mean()
            sm_climatology = soil_moisture.filterDate('1981-01-01', '2010-12-31') \
                .filter(ee.Filter.calendarRange(target_date.get('month'), target_date.get('month'), 'month')) \
                .mean()
            sm_anomaly = sm_current.subtract(sm_climatology).divide(
                soil_moisture.filterDate('1981-01-01', '2010-12-31') \
                .filter(ee.Filter.calendarRange(target_date.get('month'), target_date.get('month'), 'month')) \
                .reduce(ee.Reducer.stdDev())
            )
            
            # Composite drought index (weighted average)
            composite_drought_index = pdsi_value.multiply(0.3) \
                .add(spi_3m.multiply(0.3)) \
                .add(ndvi_anomaly.multiply(0.2)) \
                .add(sm_anomaly.multiply(0.2)) \
                .rename('composite_drought_index')
            
            # Classify drought severity
            drought_severity = ee.Image(0) \
                .where(composite_drought_index.lt(-2), 4) \
                .where(composite_drought_index.gte(-2).And(composite_drought_index.lt(-1.5)), 3) \
                .where(composite_drought_index.gte(-1.5).And(composite_drought_index.lt(-1)), 2) \
                .where(composite_drought_index.gte(-1).And(composite_drought_index.lt(-0.5)), 1) \
                .where(composite_drought_index.gte(-0.5), 0) \
                .rename('drought_severity')
            
            return ee.Image.cat([composite_drought_index, drought_severity]) \
                .set('system:time_start', target_date.millis())
        
        return {
            'calculator': calculate_drought_composite_index,
            'data_sources': {
                'pdsi': pdsi,
                'chirps': chirps,
                'modis_ndvi': modis_ndvi,
                'soil_moisture': soil_moisture
            }
        }
    
    # 2. Flood monitoring system
    def setup_flood_monitoring():
        """
        Sets up flood monitoring using SAR and optical data.
        """
        
        # Sentinel-1 SAR for flood detection
        s1_collection = ee.ImageCollection('COPERNICUS/S1_GRD')
        
        # Landsat for optical flood detection
        landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
        
        def detect_flood_event(event_date, reference_period_days=30):
            """
            Detects flood events by comparing to reference conditions.
            """
            
            event_date = ee.Date(event_date)
            reference_start = event_date.advance(-reference_period_days-7, 'day')
            reference_end = event_date.advance(-7, 'day')
            
            # SAR-based flood detection
            def sar_flood_detection():
                # Reference SAR image (median of pre-event period)
                reference_sar = s1_collection \
                    .filterBounds(region) \
                    .filterDate(reference_start, reference_end) \
                    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
                    .select('VV') \
                    .median()
                
                # Event SAR image
                event_sar = s1_collection \
                    .filterBounds(region) \
                    .filterDate(event_date, event_date.advance(3, 'day')) \
                    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
                    .select('VV') \
                    .median()
                
                # Calculate difference (flood areas show decreased backscatter)
                sar_difference = reference_sar.subtract(event_sar)
                
                # Threshold for flood detection (areas with >3dB decrease)
                flood_mask_sar = sar_difference.gt(3)
                
                return flood_mask_sar
            
            # Optical-based flood detection
            def optical_flood_detection():
                # Reference optical image
                reference_optical = landsat_collection \
                    .filterBounds(region) \
                    .filterDate(reference_start, reference_end) \
                    .map(advanced_cloud_mask) \
                    .median()
                
                # Event optical image
                event_optical = landsat_collection \
                    .filterBounds(region) \
                    .filterDate(event_date, event_date.advance(7, 'day')) \
                    .map(advanced_cloud_mask) \
                    .median()
                
                # Calculate MNDWI for both periods
                def calculate_mndwi(image):
                    green = image.select('SR_B3').multiply(0.0000275).add(-0.2)
                    swir1 = image.select('SR_B6').multiply(0.0000275).add(-0.2)
                    mndwi = green.subtract(swir1).divide(green.add(swir1))
                    return image.addBands(mndwi.rename('MNDWI'))
                
                reference_mndwi = calculate_mndwi(reference_optical).select('MNDWI')
                event_mndwi = calculate_mndwi(event_optical).select('MNDWI')
                
                # Flood detection based on MNDWI increase
                mndwi_increase = event_mndwi.subtract(reference_mndwi)
                flood_mask_optical = mndwi_increase.gt(0.2).And(event_mndwi.gt(0))
                
                return flood_mask_optical
            
            # Combine SAR and optical detections
            sar_flood = sar_flood_detection()
            optical_flood = optical_flood_detection()
            
            # Combined flood mask (either SAR or optical detection)
            combined_flood_mask = sar_flood.Or(optical_flood)
            
            # Apply topographic constraints (floods unlikely on steep slopes)
            dem = ee.Image('USGS/SRTMGL1_003')
            slope = ee.Terrain.slope(dem)
            topographic_constraint = slope.lt(10)  # Less than 10 degrees
            
            final_flood_mask = combined_flood_mask.And(topographic_constraint)
            
            return {
                'flood_mask': final_flood_mask,
                'sar_contribution': sar_flood,
                'optical_contribution': optical_flood,
                'event_date': event_date
            }
        
        return {
            'detector': detect_flood_event,
            'data_sources': {
                'sentinel1': s1_collection,
                'landsat': landsat_collection
            }
        }
    
    # Set up monitoring systems based on requested variables
    if 'drought' in monitoring_variables:
        monitoring_system['drought'] = setup_drought_monitoring()
    
    if 'flood' in monitoring_variables:
        monitoring_system['flood'] = setup_flood_monitoring()
    
    # 3. Automated alert system
    def create_alert_system(monitoring_results):
        """
        Creates automated alerts based on monitoring results.
        """
        
        alerts = []
        
        # Drought alerts
        if 'drought' in monitoring_results:
            drought_severity = monitoring_results['drought']['drought_severity']
            
            # Calculate area under different drought categories
            severe_drought_area = drought_severity.gte(3).reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=region,
                scale=1000,
                maxPixels=1e8
            )

            
            currentdate = datetime.now(timezone.utc).strftime('%Y-%m-%d')
            alerts.append({
                'type': 'drought',
                'severity': 'high' if severe_drought_area.get('drought_severity').getInfo() > 100 else 'moderate',
                'affected_area_km2': ee.Number(severe_drought_area.get('drought_severity')).multiply(1e-6).getInfo(),
                'timestamp': ee.Date(currentdate).format('YYYY-MM-dd HH:mm:ss').getInfo()
            })
        
        # Flood alerts
        if 'flood' in monitoring_results:
            flood_mask = monitoring_results['flood']['flood_mask']
            
            flood_area = flood_mask.reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=region,
                scale=30,
                maxPixels=1e8
            )
            
            flood_area_km2 = ee.Number(flood_area.get('flood_mask')).multiply(9e-7).getInfo()
            currentdate = datetime.now(timezone.utc).strftime('%Y-%m-%d')
            if flood_area_km2 > 0.1:  # Alert if flood area > 0.1 km²
                alerts.append({
                    'type': 'flood',
                    'severity': 'high' if flood_area_km2 > 10 else 'moderate',
                    'affected_area_km2': flood_area_km2,
                    'timestamp': ee.Date(currentdate).format('YYYY-MM-dd HH:mm:ss').getInfo()
                })
        
        return alerts
    
    monitoring_system['alert_system'] = create_alert_system
    
    return monitoring_system

In [7]:
def create_operational_monitoring_pipeline(region, output_configuration):
    """
    Creates a complete operational monitoring pipeline for environmental agencies.
    """
    
    def setup_automated_processing_pipeline():
        """
        Sets up automated processing pipeline with scheduling and data management.
        """
        
        pipeline_config = {
            'processing_schedule': {
                'surface_water': 'weekly',
                'vegetation_health': 'bi-weekly', 
                'climate_variables': 'monthly',
                'extreme_events': 'daily'
            },
            'data_retention': {
                'raw_data': '1_year',
                'processed_products': '10_years',
                'alerts': 'permanent'
            },
            'quality_control': {
                'cloud_threshold': 20,  # Maximum cloud cover percentage
                'data_completeness_threshold': 80,  # Minimum data availability
                'spatial_completeness_threshold': 90  # Minimum spatial coverage
            }
        }
        
        def create_processing_task(analysis_type, schedule):
            """
            Creates processing tasks for different analysis types.
            """
            
            if analysis_type == 'surface_water':
                def process_surface_water():
                    currentdate = datetime.now(timezone.utc).strftime('%Y-%m-%d')
                    current_date = ee.Date(currentdate)
                    analysis_period = (
                        current_date.advance(-7, 'day').format('YYYY-MM-dd'),
                        current_date.format('YYYY-MM-dd')
                    )
                    
                    water_results = comprehensive_surface_water_mapping(
                        region, analysis_period
                    )
                    
                    # Export results
                    export_task = ee.batch.Export.image.toDrive(
                        image=water_results['water_mask'],
                        description=f'surface_water_{current_date.format("YYYY_MM_dd").getInfo()}',
                        scale=30,
                        region=region,
                        maxPixels=1e9
                    )
                    
                    return export_task
                
                return process_surface_water
            
            elif analysis_type == 'climate_variables':
                def process_climate_data():
                    currentdate = datetime.now(timezone.utc).strftime('%Y-%m-%d')
                    current_date = ee.Date(currentdate)
                    analysis_period = (
                        current_date.advance(-1, 'month').format('YYYY-MM-dd'),
                        current_date.format('YYYY-MM-dd')
                    )
                    
                    climate_results = comprehensive_climate_analysis(
                        region, analysis_period
                    )
                    
                    # Export multiple climate variables
                    export_tasks = []
                    for source_name, source_data in climate_results['climate_data_sources'].items():
                        for var_name in source_data['variables'].keys():
                            if var_name in ['temperature_max', 'temperature_min', 'precipitation']:
                                monthly_composite = source_data['collection'].median()
                                
                                export_task = ee.batch.Export.image.toDrive(
                                    image=monthly_composite.select(source_data['variables'][var_name]),
                                    description=f'{var_name}_{source_name}_{current_date.format("YYYY_MM").getInfo()}',
                                    scale=5000,
                                    region=region,
                                    maxPixels=1e9
                                )
                                
                                export_tasks.append(export_task)
                    
                    return export_tasks
                
                return process_climate_data
            
            # Add more processing functions for other analysis types
            return None
        
        # Create processing tasks
        processing_tasks = {}
        for analysis_type, schedule in pipeline_config['processing_schedule'].items():
            processing_tasks[analysis_type] = {
                'function': create_processing_task(analysis_type, schedule),
                'schedule': schedule,
                'last_run': None,
                'next_run': None
            }
        
        return {
            'config': pipeline_config,
            'tasks': processing_tasks
        }
    
    def setup_data_management_system():
        """
        Sets up data management system with versioning and metadata.
        """
        
        data_management = {
            'asset_naming_convention': {
                'prefix': f"environmental_monitoring_{region.getInfo()['type']}",
                'date_format': 'YYYY_MM_dd',
                'version_format': 'v{major}_{minor}_{patch}'
            },
            'metadata_schema': {
                'required_fields': [
                    'processing_date', 'data_source', 'analysis_type',
                    'spatial_resolution', 'temporal_coverage', 'quality_score'
                ],
                'optional_fields': [
                    'cloud_coverage', 'data_gaps', 'processing_parameters',
                    'accuracy_assessment', 'validation_results'
                ]
            },
            'quality_assurance': {
                'automated_checks': [
                    'spatial_completeness', 'temporal_consistency',
                    'value_ranges', 'metadata_completeness'
                ],
                'manual_validation': [
                    'accuracy_assessment', 'anomaly_investigation',
                    'user_feedback_integration'
                ]
            }
        }
        
        def generate_product_metadata(image, analysis_type, processing_params):
            """
            Generates comprehensive metadata for processed products.
            """
            currentdate = datetime.now(timezone.utc).strftime('%Y-%m-%d')
            metadata = {
                'product_id': ee.String(data_management['asset_naming_convention']['prefix']) \
                    .cat('_').cat(ee.Date(currentdate).format('YYYY_MM_dd')) \
                    .cat('_').cat(analysis_type),
                'processing_date': ee.Date(currentdate).format('YYYY-MM-dd HH:mm:ss'),
                'analysis_type': analysis_type,
                'spatial_bounds': region.bounds().getInfo(),
                'processing_parameters': processing_params,
                'data_quality_score': image.get('quality_score', -1)
            }
            
            return metadata
        
        return {
            'config': data_management,
            'metadata_generator': generate_product_metadata
        }
    
    def setup_monitoring_dashboard():
        """
        Sets up monitoring dashboard configuration for real-time status tracking.
        """
        
        dashboard_config = {
            'update_frequency': 'hourly',
            'key_indicators': [
                'current_water_extent',
                'drought_severity_level',
                'extreme_event_alerts',
                'data_processing_status',
                'system_health_metrics'
            ],
            'visualization_types': {
                'time_series_charts': ['precipitation', 'temperature', 'water_extent'],
                'maps': ['current_conditions', 'anomalies', 'alerts'],
                'statistics': ['summary_stats', 'trend_indicators', 'quality_metrics']
            },
            'alert_thresholds': {
                'drought_severity': {'moderate': 2, 'severe': 3, 'extreme': 4},
                'flood_area_km2': {'moderate': 1, 'severe': 10, 'extreme': 50},
                'data_latency_hours': {'warning': 6, 'critical': 24},
                'processing_failure_rate': {'warning': 0.05, 'critical': 0.1}
            }
        }
        
        def generate_dashboard_data():
            """
            Generates data for the monitoring dashboard.
            """
            currentdate = datetime.now(timezone.utc).strftime('%Y-%m-%d')
            current_date = ee.Date(currentdate)
            
            # Current status indicators
            status_indicators = {
                'last_update': current_date.format('YYYY-MM-dd HH:mm:ss'),
                'system_status': 'operational',  # operational, maintenance, error
                'active_alerts': [],
                'data_freshness': {}
            }
            
            # Generate summary statistics
            def calculate_summary_statistics():
                """Calculate key summary statistics for dashboard."""
                
                # Recent water extent
                recent_water = comprehensive_surface_water_mapping(
                    region, 
                    (current_date.advance(-7, 'day').format('YYYY-MM-dd'), 
                     current_date.format('YYYY-MM-dd'))
                )
                
                water_extent_km2 = recent_water['water_mask'].reduceRegion(
                    reducer=ee.Reducer.sum(),
                    geometry=region,
                    scale=30,
                    maxPixels=1e9
                ).get('water_mask')
                
                # Recent climate conditions
                recent_climate = comprehensive_climate_analysis(
                    region,
                    (current_date.advance(-30, 'day').format('YYYY-MM-dd'),
                     current_date.format('YYYY-MM-dd'))
                )
                
                summary_stats = {
                    'water_extent_km2': ee.Number(water_extent_km2).multiply(9e-7).getInfo(),
                    'region_area_km2': region.area().divide(1e6).getInfo(),
                    'water_percentage': ee.Number(water_extent_km2).multiply(9e-7).divide(region.area().divide(1e6)).multiply(100).getInfo()
                }
                
                return summary_stats
            
            summary_stats = calculate_summary_statistics()
            status_indicators['summary_statistics'] = summary_stats
            
            return status_indicators
        
        return {
            'config': dashboard_config,
            'data_generator': generate_dashboard_data
        }
    
    # Integration with external systems
    def setup_external_integrations():
        """
        Sets up integrations with external systems and APIs.
        """
        
        integrations = {
            'weather_services': {
                'national_weather_service': {
                    'api_endpoint': 'https://api.weather.gov/',
                    'data_types': ['forecasts', 'warnings', 'observations'],
                    'update_frequency': 'hourly'
                },
                'global_forecast_system': {
                    'data_source': 'NOAA/GFS0P25',
                    'variables': ['temperature_2m', 'precipitation_surface', 'relative_humidity_2m'],
                    'forecast_horizon': '7_days'
                }
            },
            'hydrological_services': {
                'usgs_streamflow': {
                    'api_endpoint': 'https://waterservices.usgs.gov/nwis/iv/',
                    'parameters': ['discharge', 'gage_height', 'temperature'],
                    'stations': []  # Would be populated with relevant station IDs
                },
                'reservoir_levels': {
                    'data_source': 'reservoir_monitoring_network',
                    'update_frequency': 'daily'
                }
            },
            'emergency_management': {
                'alert_distribution': {
                    'email_notifications': True,
                    'sms_alerts': True,
                    'web_services': ['fema_integration', 'state_emergency_management'],
                    'social_media': ['twitter_api', 'emergency_broadcast']
                }
            }
        }
        
        def create_data_fusion_workflow(gee_results, external_data):
            """
            Creates workflow to fuse GEE results with external data sources.
            """
            
            fused_analysis = {}
            
            # Combine GEE water extent with USGS streamflow data
            if 'surface_water' in gee_results and 'usgs_streamflow' in external_data:
                # This would involve API calls to USGS and correlation analysis
                # Simplified representation here
                fused_analysis['water_flow_correlation'] = {
                    'gee_water_extent': gee_results['surface_water']['water_extent_km2'],
                    'streamflow_data': external_data['usgs_streamflow'],
                    'correlation_analysis': 'to_be_implemented'
                }
            
            # Combine GEE drought indicators with agricultural reports
            if 'drought_monitoring' in gee_results:
                fused_analysis['agricultural_impact'] = {
                    'drought_severity': gee_results['drought_monitoring']['composite_drought_index'],
                    'crop_condition_reports': external_data.get('agricultural_reports', {}),
                    'economic_impact_estimate': 'to_be_calculated'
                }
            
            return fused_analysis
        
        return {
            'config': integrations,
            'fusion_workflow': create_data_fusion_workflow
        }
    
    # Assemble complete operational pipeline
    currentdate = datetime.now(timezone.utc).strftime('%Y-%m-%d')
    operational_pipeline = {
        'processing_pipeline': setup_automated_processing_pipeline(),
        'data_management': setup_data_management_system(),
        'monitoring_dashboard': setup_monitoring_dashboard(),
        'external_integrations': setup_external_integrations(),
        'configuration': {
            'region': region,
            'output_config': output_configuration,
            'deployment_date': ee.Date(currentdate).format('YYYY-MM-dd').getInfo(),
            'version': '1.0.0'
        }
    }
    return operational_pipeline 

In [8]:
# Example: Lake Tahoe region (polygon coordinates)
region = ee.Geometry.Polygon(
[[[-120.00, 39.00],
[-119.90, 39.00],
[-119.90, 39.10],
[-120.00, 39.10],
[-120.00, 39.00]]])

print(f"Region of interest defined: {region.getInfo()}")

# --- Running Examples ---

# Example 0: Comprehensive Climate Analysis
print("\n--- Running Comprehensive Climate Analysis ---")
analysis_period_climate = ('2020-01-01', '2021-01-01')
climate_results = comprehensive_climate_analysis(region, analysis_period_climate)
print("Climate Analysis Results (partial view):")
print(climate_results.keys())

# Example 1: Create Operational Monitoring Pipeline
print("\n--- Running Operational Monitoring Pipeline Example ---")
output_config_pipeline = {"export_folder": "GEE_Monitoring_Outputs"}
operational_pipeline = create_operational_monitoring_pipeline(region, output_config_pipeline)
print("Operational Pipeline Setup (partial view):")
print(operational_pipeline["processing_pipeline"]["tasks"].keys())

# Example 2: Create Extreme Event Monitoring System
print("\n--- Running Extreme Event Monitoring System Example ---")
monitoring_vars = ["drought", "flood"] # Limiting for a quicker example
extreme_event_system = create_extreme_event_monitoring_system(region, monitoring_vars)
print("Extreme Event Monitoring System Setup (partial view):")
print(extreme_event_system.keys())


Region of interest defined: {'type': 'Polygon', 'coordinates': [[[-120, 39], [-119.9, 39], [-119.9, 39.1], [-120, 39.1], [-120, 39]]]}

--- Running Comprehensive Climate Analysis ---
Climate Analysis Results (partial view):
dict_keys(['climate_data_sources', 'trend_analysis', 'extreme_analysis', 'impact_assessment'])

--- Running Operational Monitoring Pipeline Example ---
Operational Pipeline Setup (partial view):
dict_keys(['surface_water', 'vegetation_health', 'climate_variables', 'extreme_events'])

--- Running Extreme Event Monitoring System Example ---
Extreme Event Monitoring System Setup (partial view):
dict_keys(['drought', 'flood', 'alert_system'])


C:\Users\colla\AppData\Local\Programs\Python\Python313\Lib\site-packages\ee\deprecation.py:209: DeprecationWarning: 

Attention required for MODIS/006/MOD13A1! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD13A1

  warnings.warn(warning, category=DeprecationWarning)


In [10]:
extreme_event_system

{'drought': {'calculator': <function __main__.create_extreme_event_monitoring_system.<locals>.setup_drought_monitoring.<locals>.calculate_drought_composite_index(date)>,
  'data_sources': {'pdsi': <ee.imagecollection.ImageCollection at 0x23229756630>,
   'chirps': <ee.imagecollection.ImageCollection at 0x23229756810>,
   'modis_ndvi': <ee.imagecollection.ImageCollection at 0x23229757410>,
   'soil_moisture': <ee.imagecollection.ImageCollection at 0x232297575f0>}},
 'flood': {'detector': <function __main__.create_extreme_event_monitoring_system.<locals>.setup_flood_monitoring.<locals>.detect_flood_event(event_date, reference_period_days=30)>,
  'data_sources': {'sentinel1': <ee.imagecollection.ImageCollection at 0x23229757710>,
   'landsat': <ee.imagecollection.ImageCollection at 0x23229757650>}},
 'alert_system': <function __main__.create_extreme_event_monitoring_system.<locals>.create_alert_system(monitoring_results)>}

In [11]:
import pandas as pd

def ic_to_dataframe(ic_dict, source, variable, region, scale=10000):
    """
    Convert an ImageCollection in your dictionary to a Pandas DataFrame.

    Parameters:
        ic_dict (dict): The full dictionary with ImageCollection references.
        source (str): e.g., 'era5_monthly', 'terraclimate', 'gridmet'.
        variable (str): e.g., 'temperature_2m', 'precipitation'.
        region (ee.Geometry): Earth Engine geometry to sample.
        scale (int): Spatial resolution in meters.

    Returns:
        pd.DataFrame: columns=['date', 'value'].
    """
    # Get the ImageCollection and the corresponding band
    collection = ic_dict['climate_data_sources'][source]['collection']
    band_name = ic_dict['climate_data_sources'][source]['variables'][variable]

    # Convert collection to a list
    ic_list = collection.toList(collection.size())
    data = []

    for i in range(collection.size().getInfo()):
        img = ee.Image(ic_list.get(i))
        # print(img.getInfo())
        date = img.date().format('YYYY-MM-dd').getInfo()
        value = img.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=region,
            scale=scale
        ).get(band_name).getInfo()
        data.append({'date': date, 'value': value})

    return pd.DataFrame(data)

df = ic_to_dataframe(climate_results, "era5_monthly", "temperature_2m", region, scale=500)
df.head(10)

In [12]:
df = ic_to_dataframe(climate_results, "era5_monthly", "temperature_2m", region, scale=500)
df.head(10)
#note temperature are in K

,date,value
0,2020-01-01,273.884583
1,2020-02-01,274.092499
2,2020-03-01,273.984650
3,2020-04-01,278.569580
4,2020-05-01,283.736420
5,2020-06-01,287.967224


In [13]:
def implement_advanced_solutions():
    """
    Implements solutions for common challenges in operational Earth observation.
    """
    
    # 1. Data gap handling and interpolation
    def handle_data_gaps(image_collection, max_gap_days=30):
        """
        Handles data gaps using spatiotemporal interpolation methods.
        """
        
        def interpolate_temporal_gaps(collection):
            """Interpolates missing values in time series."""
            
            # Convert collection to list for gap detection
            image_list = collection.toList(collection.size())
            dates_list = collection.aggregate_array('system:time_start')
            
            def fill_gap(current_index):
                """Fill gap at current index using neighboring images."""
                
                current_image = ee.Image(image_list.get(current_index))
                current_date = ee.Date(current_image.get('system:time_start'))
                
                # Find previous and next valid images
                prev_image = ee.Image(image_list.get(ee.Number(current_index).subtract(1).max(0)))
                next_image = ee.Image(image_list.get(ee.Number(current_index).add(1).min(collection.size().subtract(1))))
                
                # Temporal interpolation
                prev_date = ee.Date(prev_image.get('system:time_start'))
                next_date = ee.Date(next_image.get('system:time_start'))
                
                # Calculate interpolation weights
                total_gap = next_date.difference(prev_date, 'day')
                weight_next = current_date.difference(prev_date, 'day').divide(total_gap)
                weight_prev = ee.Number(1).subtract(weight_next)
                
                # Weighted interpolation
                interpolated = prev_image.multiply(weight_prev).add(next_image.multiply(weight_next))
                
                return interpolated.set('system:time_start', current_date.millis())
                    .set('interpolated', True)
            
            # Apply interpolation to identified gaps
            # This is a simplified version - production systems would use more sophisticated gap detection
            return collection
        
        def spatial_interpolation(image, reference_images):
            """Fills spatial gaps using neighboring pixels and reference data."""
            
            # Identify missing pixels
            valid_pixels = image.mask()
            
            # Use focal mean for spatial interpolation
            spatial_fill = image.focalMean(radius=3, kernelType='circle', units='pixels')
            
            # Combine original and interpolated values
            filled_image = image.where(valid_pixels.eq(0), spatial_fill)
            
            return filled_image.set('spatial_interpolation_applied', True)
        
        return {
            'temporal_interpolation': interpolate_temporal_gaps,
            'spatial_interpolation': spatial_interpolation
        }
    
    # 2. Multi-sensor calibration and harmonization
    def implement_sensor_harmonization():
        """
        Implements cross-sensor calibration and harmonization methods.
        """
        
        def harmonize_landsat_sentinel(landsat_image, sentinel_image):
            """
            Harmonizes Landsat and Sentinel-2 reflectance values.
            """
            
            # Landsat 8 to Sentinel-2 harmonization coefficients
            # Based on published cross-calibration studies
            harmonization_coefficients = {
                'blue': {'slope': 0.9959, 'intercept': -0.0002},
                'green': {'slope': 0.9778, 'intercept': -0.004},
                'red': {'slope': 0.9837, 'intercept': -0.0023},
                'nir': {'slope': 0.9683, 'intercept': 0.0084}
            }
            
            def apply_harmonization(image, sensor_type):
                """Apply harmonization coefficients to image."""
                
                if sensor_type == 'landsat_to_sentinel':
                    # Convert Landsat to Sentinel-2 equivalent
                    blue_harm = image.select('SR_B2').multiply(0.0000275).add(-0.2) \
                        .multiply(harmonization_coefficients['blue']['slope']) \
                        .add(harmonization_coefficients['blue']['intercept'])
                    
                    green_harm = image.select('SR_B3').multiply(0.0000275).add(-0.2) \
                        .multiply(harmonization_coefficients['green']['slope']) \
                        .add(harmonization_coefficients['green']['intercept'])
                    
                    red_harm = image.select('SR_B4').multiply(0.0000275).add(-0.2) \
                        .multiply(harmonization_coefficients['red']['slope']) \
                        .add(harmonization_coefficients['red']['intercept'])
                    
                    nir_harm = image.select('SR_B5').multiply(0.0000275).add(-0.2) \
                        .multiply(harmonization_coefficients['nir']['slope']) \
                        .add(harmonization_coefficients['nir']['intercept'])
                    
                    return ee.Image.cat([blue_harm, green_harm, red_harm, nir_harm]) \
                        .rename(['blue', 'green', 'red', 'nir']) \
                        .set('harmonized', True, 'source_sensor', 'landsat_harmonized_to_sentinel')
                
                return image
            
            harmonized_landsat = apply_harmonization(landsat_image, 'landsat_to_sentinel')
            
            return {
                'harmonized_landsat': harmonized_landsat,
                'original_sentinel': sentinel_image,
                'harmonization_applied': True
            }
        
        return harmonize_landsat_sentinel
    
    # 3. Uncertainty quantification and propagation
    def implement_uncertainty_analysis():
        """
        Implements comprehensive uncertainty analysis for Earth observation products.
        """
        
        def calculate_measurement_uncertainty(image, sensor_specs):
            """
            Calculates measurement uncertainty based on sensor specifications.
            """
            
            # Sensor-specific uncertainty parameters
            uncertainty_params = {
                'landsat8': {
                    'radiometric_uncertainty': 3,  # %
                    'geometric_uncertainty': 12,   # meters
                    'spectral_uncertainty': 1      # %
                },
                'sentinel2': {
                    'radiometric_uncertainty': 5,  # %
                    'geometric_uncertainty': 20,   # meters  
                    'spectral_uncertainty': 2      # %
                }
            }
            
            sensor_type = sensor_specs.get('sensor_type', 'landsat8')
            params = uncertainty_params[sensor_type]
            
            # Calculate pixel-wise uncertainty
            radiometric_uncertainty = image.multiply(params['radiometric_uncertainty'] / 100.0)
            
            # Geometric uncertainty affects spatial products
            geometric_uncertainty_map = ee.Image.constant(params['geometric_uncertainty'])
            
            return {
                'radiometric_uncertainty': radiometric_uncertainty,
                'geometric_uncertainty': geometric_uncertainty_map,
                'combined_uncertainty': radiometric_uncertainty.hypot(
                    ee.Image.constant(params['spectral_uncertainty'] / 100.0)
                )
            }
        
        def propagate_uncertainty_through_analysis(input_uncertainties, analysis_function):
            """
            Propagates uncertainty through analytical workflows using Monte Carlo methods.
            """
            
            # Simplified Monte Carlo uncertainty propagation
            n_iterations = 100
            
            def monte_carlo_iteration(iteration):
                """Single Monte Carlo iteration with perturbed inputs."""
                
                # Add random noise based on uncertainty estimates
                noise_factor = ee.Number.random().subtract(0.5).multiply(2)  # -1 to 1
                perturbed_input = input_uncertainties['input_image'].add(
                    input_uncertainties['uncertainty_image'].multiply(noise_factor)
                )
                
                # Apply analysis function to perturbed input
                result = analysis_function(perturbed_input)
                
                return result.set('iteration', iteration)
            
            # Run Monte Carlo iterations
            iterations = ee.List.sequence(0, n_iterations-1)
            monte_carlo_results = iterations.map(monte_carlo_iteration)
            
            # Calculate uncertainty statistics
            results_collection = ee.ImageCollection.fromImages(monte_carlo_results)
            mean_result = results_collection.mean()
            std_result = results_collection.reduce(ee.Reducer.stdDev())
            
            return {
                'mean_result': mean_result,
                'uncertainty_estimate': std_result,
                'confidence_interval': {
                    'lower_95': results_collection.reduce(ee.Reducer.percentile([2.5])),
                    'upper_95': results_collection.reduce(ee.Reducer.percentile([97.5]))
                }
            }
        
        return {
            'measurement_uncertainty': calculate_measurement_uncertainty,
            'uncertainty_propagation': propagate_uncertainty_through_analysis
        }
    
    return {
        'data_gap_handling': handle_data_gaps,
        'sensor_harmonization': implement_sensor_harmonization(),
        'uncertainty_analysis': implement_uncertainty_analysis()
    }

IndentationError: unexpected indent (1908881956.py, line 42)